In [ ]:
import sys

sys.path.append('../../..')

In [2]:
from batchflow.research import Research

def power(a=2, b=3):
    return a ** b

research = Research().add_callable(power, save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 1/1 [00:00<00:00,  4.19it/s]


,id,iteration,power
0,b8c9f03148557255,0,8


In [3]:
research = Research().add_callable(power, a=3, b=2, save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 1/1 [00:00<00:00,  3.33it/s]


,id,iteration,power
0,b8c9f03146175177,0,9


In [4]:
research = Research().add_callable(power, a=3, b=2, save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 1/1 [00:00<00:00,  2.63it/s]


,id,iteration,power
0,b8c9f03113247980,0,9


In [5]:
research = Research().power(3, 2, save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 1/1 [00:00<00:00,  2.09it/s]


,id,iteration,power
0,b8c9f03183769365,0,9


In [6]:
from batchflow.research import Domain, EC

domain = Domain({'a': [2, 3], 'b': [2, 3, 4]})

research = Research(domain=domain).power(a=EC('a'), b=EC('b'), save_to='power')
research.run(dump_results=False)

research.results.df

0it [00:02, ?it/s]
0it [00:02, ?it/s]
0it [00:01, ?it/s]
0it [00:01, ?it/s]
0it [00:03, ?it/s]/6 [00:00<00:01,  3.28it/s]
0it [00:02, ?it/s]
0it [00:02, ?it/s]
100%|██████████| 6/6 [00:01<00:00,  4.51it/s]


,id,a,b,iteration,power
0,bef0db4979408160,2,2,0,4
1,984d62c954860355,2,3,0,8
2,270aa80374753129,2,4,0,16
3,5420406317860656,3,2,0,9
4,72b9965c55235311,3,3,0,27
5,4b4a714554598765,3,4,0,81


In [7]:
from batchflow.research import Domain, EC

domain = Domain({'a': [2, 3]}) * Domain({'b': [2, 3, 4]})

research = Research(domain=domain).power(a=EC('a'), b=EC('b'), save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 6/6 [00:00<00:00,  8.43it/s]


,id,a,b,iteration,power
0,bef0db4955026963,2,2,0,4
1,984d62c944622284,2,3,0,8
2,270aa80300467580,2,4,0,16
3,5420406370824287,3,2,0,9
4,72b9965c40785381,3,3,0,27
5,4b4a714518660346,3,4,0,81


In [8]:
domain = Domain({'a': [2, 3]}) + Domain({'b': [2, 3, 4]})

research = Research(domain=domain).power(kwargs=EC(), save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 5/5 [00:00<00:00,  6.14it/s]


,id,a,b,iteration,power
0,71d4879f40250182,2.0,NaN,0,8
1,e32d7dce15463302,3.0,NaN,0,27
2,ff973b7631662410,NaN,2.0,0,4
3,6bffa3ee67484965,NaN,3.0,0,8
4,05064ba246174379,NaN,4.0,0,16


In [9]:
from batchflow.research import O

domain = Domain({'start': [2, 3], 'b': [3], 'length': [1, 2]})

def sequence(start, length):
    end = start + length
    return (i for i in range(start, end))

research = (Research(domain=domain)
            .add_generator(sequence, start=EC('start'), length=EC('length'))
            .add_callable(power, a=O('sequence'), b=EC('b'), save_to='power')
           )
research.run(dump_results=False)

research.results.df

100%|██████████| 4/4 [00:00<00:00,  4.51it/s]


,id,start,b,length,iteration,power
0,71be4bbd64769239,2,3,1,0,8
1,71be4bbd64769239,2,3,1,1,8
2,2b80695214919307,2,3,2,0,8
3,2b80695214919307,2,3,2,1,27
4,2b80695214919307,2,3,2,2,27
5,23d6441411272446,3,3,1,0,27
6,23d6441411272446,3,3,1,1,27
7,77b8363f18999211,3,3,2,0,27
8,77b8363f18999211,3,3,2,1,64
9,77b8363f18999211,3,3,2,2,64


In [10]:
research = (Research(domain=domain)
            .add_generator(sequence, start=EC('start'), length=EC('length'))
            .add_callable(power, a=O('sequence'), b=EC('b'), save_to='power')
            .add_callable('inc', lambda x: x + 1, x=O('power'), save_to='inc', when='last')
           )

research.run(dump_results=False, n_iters=2)

research.results.df

100%|██████████| 8/8 [00:01<00:00,  7.80it/s]


,id,start,b,length,iteration,power,inc
0,71be4bbd47111984,2,3,1,0,8,NaN
1,71be4bbd47111984,2,3,1,1,8,9.0
2,2b80695282323566,2,3,2,0,8,NaN
3,2b80695282323566,2,3,2,1,27,28.0
4,23d6441488354762,3,3,1,0,27,NaN
5,23d6441488354762,3,3,1,1,27,28.0
6,77b8363f83601188,3,3,2,0,27,NaN
7,77b8363f83601188,3,3,2,1,64,65.0


In [11]:
research.results.to_df(pivot=False)

,id,start,b,length,iteration,name,value
0,71be4bbd47111984,2,3,1,0,power,8
1,71be4bbd47111984,2,3,1,1,power,8
2,71be4bbd47111984,2,3,1,1,inc,9
3,2b80695282323566,2,3,2,0,power,8
4,2b80695282323566,2,3,2,1,power,27
5,2b80695282323566,2,3,2,1,inc,28
6,23d6441488354762,3,3,1,0,power,27
7,23d6441488354762,3,3,1,1,power,27
8,23d6441488354762,3,3,1,1,inc,28
9,77b8363f83601188,3,3,2,0,power,27


In [12]:
class MyCalc:
    def __init__(self, b):
        self.b = b
    
    def power(self, a):
        return a ** self.b

research = (Research(domain=domain)
            .add_instance('calc', MyCalc, b=EC('b'))
            .add_generator(sequence, start=EC('start'), length=EC('length'))
            .add_callable('calc.power', a=O('sequence'), save_to='power')
           )

research.run(n_iters=2, dump_results=False)

research.results.df

100%|██████████| 8/8 [00:01<00:00,  7.16it/s]


,id,start,b,length,iteration,power
0,71be4bbd87481027,2,3,1,0,8
1,71be4bbd87481027,2,3,1,1,8
2,2b80695227396687,2,3,2,0,8
3,2b80695227396687,2,3,2,1,27
4,23d6441426370831,3,3,1,0,27
5,23d6441426370831,3,3,1,1,27
6,77b8363f53236862,3,3,2,0,27
7,77b8363f53236862,3,3,2,1,64


In [13]:
def heavy_callable():
    i = int(2 * 10e1)
    for i in range(i):
        pass

In [14]:
%%timeit

research = Research(n_reps=2).add_callable(heavy_callable)
research.run(dump_results=False)

100%|██████████| 2/2 [00:00<00:00,  2.35it/s]
0it [00:01, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  2.09it/s]
0it [00:02, ?it/s]
0it [00:01, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  2.04it/s]
0it [00:02, ?it/s]
0it [00:01, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  2.00it/s]
0it [00:02, ?it/s]
0it [00:01, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  2.30it/s]


1.1 s ± 62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit

research = Research(n_reps=2).add_callable(heavy_callable)
research.run(dump_results=False, workers=2)

100%|██████████| 2/2 [00:00<00:00,  2.55it/s]
0it [00:19, ?it/s]
0it [00:18, ?it/s]
0it [00:17, ?it/s]
0it [00:16, ?it/s]
0it [00:02, ?it/s]
0it [00:01, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  3.98it/s]
0it [00:01, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  4.15it/s]
0it [00:01, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  3.87it/s]
0it [00:01, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  4.86it/s]


598 ms ± 40.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
import numpy as np
from batchflow.research import E

def load_data(random):
    return random.normal(size=10)

def mean(array, use_numpy=True):
    if use_numpy:
        return f"{np.mean(array):.02f} (with numpy)"
    return f"{sum(array) / len(array):.02f} (without numpy)"

domain = Domain({'use_numpy': [False, True]})

research = (Research(domain=domain)
            .add_callable(load_data, random=E().random)
            .add_callable(mean, array=O('load_data'), use_numpy=EC('use_numpy'),
                          save_to='stat')
           )

research.remove('research', ask=False)
research.run(n_iters=2)

research.results.df

0it [00:00, ?it/s]
100%|██████████| 4/4 [00:00<00:00,  7.55it/s]


,id,use_numpy,iteration,stat
0,7635c7e421299804,False,0,-0.11 (without numpy)
1,7635c7e421299804,False,1,0.00 (without numpy)
2,8e95395722131143,True,0,-0.14 (with numpy)
3,8e95395722131143,True,1,0.63 (with numpy)


In [17]:
research = (Research(domain=domain)
            .add_callable(load_data, random=E().random, root=True)
            .add_callable(mean, array=O('load_data'), use_numpy=EC('use_numpy'),
                          save_to='stat')
           )

research.remove('research', ask=False)
research.run(n_iters=2, branches=2)

research.results.df

100%|██████████| 4/4 [00:00<00:00,  9.02it/s]


,id,use_numpy,iteration,stat
0,7635c7e456256606,False,0,0.11 (without numpy)
1,7635c7e456256606,False,1,-0.11 (without numpy)
2,8e95395727709467,True,0,0.11 (with numpy)
3,8e95395727709467,True,1,-0.11 (with numpy)
